In [1]:
import os
import sys
import joblib
import torch
import trimesh
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append('/home/sid/Projects/reprojection')
sys.path.append('/home/sid/Projects/humor/humor')

from body_model.body_model import BodyModel

In [2]:
def visualize_coordinate_system(mesh, transform_option='identity', save_path=None):
    """
    Visualize mesh with different coordinate transformations.
    
    Args:
        mesh (trimesh.Trimesh): Input mesh to visualize
        transform_option (str): Transformation to apply:
            - 'identity': No transformation
            - 'invert_yz': Invert Y and Z axes
            - 'invert_xz': Invert X and Z axes
            - 'rotate_y_180': Rotate 180° around Y
            - 'swap_yz': Swap Y and Z axes
            - 'invert_z': Invert Z-axis
        save_path (str, optional): Path to save visualization
    """
    # Create transformation matrix
    if transform_option == 'identity':
        alignment_matrix = np.eye(4)
    elif transform_option == 'invert_yz':
        alignment_matrix = np.eye(4)
        alignment_matrix[1, 1] = -1
        alignment_matrix[2, 2] = -1
    elif transform_option == 'invert_xz':
        alignment_matrix = np.eye(4)
        alignment_matrix[0, 0] = -1
        alignment_matrix[2, 2] = -1
    elif transform_option == 'rotate_y_180':
        alignment_matrix = trimesh.transformations.rotation_matrix(np.pi, [0, 1, 0])
    elif transform_option == 'swap_yz':
        alignment_matrix = np.array([
            [1, 0, 0, 0],
            [0, 0, 1, 0],
            [0, 1, 0, 0],
            [0, 0, 0, 1],
        ])
    elif transform_option == 'invert_z':
        alignment_matrix = np.eye(4)
        alignment_matrix[2, 2] = -1
    else:
        raise ValueError(f"Unknown option '{transform_option}'")

    # Apply transformation
    mesh = mesh.copy()
    mesh.apply_transform(alignment_matrix)

    # Visualization
    fig = plt.figure(figsize=(15, 5))
    views = [(0, 0), (90, 0), (0, 90)]
    titles = ['Front (X-Y)', 'Side (Y-Z)', 'Top (X-Z)']
    
    for i, (elev, azim) in enumerate(views, 1):
        ax = fig.add_subplot(1, 3, i, projection='3d')
        
        # Plot vertices
        vertices = mesh.vertices
        ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], 
                  c='b', marker='.', s=1, alpha=0.1)
        
        # Plot axes
        origin = mesh.centroid
        axis_length = np.max(np.ptp(vertices, axis=0)) * 0.2
        
        ax.quiver(origin[0], origin[1], origin[2], 
                 axis_length, 0, 0, color='r', label='X')
        ax.quiver(origin[0], origin[1], origin[2], 
                 0, axis_length, 0, color='g', label='Y')
        ax.quiver(origin[0], origin[1], origin[2], 
                 0, 0, axis_length, color='b', label='Z')
        
        ax.set_title(f"{titles[i-1]}\n{transform_option}")
        ax.view_init(elev=elev, azim=azim)
        ax.legend()
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()
    
    # Print mesh statistics
    print(f"\nMesh Statistics ({transform_option}):")
    print(f"Centroid: {mesh.centroid}")
    print(f"Bounds: {mesh.bounds}")
    print(f"Up vector (Z) mean: {mesh.vertices[:, 2].mean():.3f}")
    return mesh


In [ ]:
HUMOR_OUT_FILE = "/home/NAS-mountpoint/kinect-omni-ego/2023-02-09/at-unis/lab/a04-2/capture0/out_capture0/results_out/final_results/stage3_results.npz"
WHAM_OUT_FILE = '/home/sid/Projects/WHAM/output/demo/2023-02-09_at-unis_lab_a01_capture0/wham_output.pkl'
BM_PATH = "/home/sid/Projects/humor/body_models/smplh/male/model.npz"

humor_output = np.load(HUMOR_OUT_FILE)
wham_output = joblib.load(WHAM_OUT_FILE)

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize BodyModel with 22 betas
bm = BodyModel(
    bm_path=BM_PATH,
    num_betas=22,  # Changed to match SMPLH shape space
    batch_size=1,
    model_type='smplh'
).to(device)

# Load WHAM data and pad betas to 22
wham_betas = torch.tensor(wham_data[0]["betas"][0], device=device).float()  # [10]
wham_betas = torch.cat([
    wham_betas,
    torch.zeros(12, device=device)  # Pad to 22 dims
])

# Rest of WHAM processing
wham_pose = torch.tensor(wham_data[0]["pose_world"][0], device=device).float()
wham_trans = torch.tensor(wham_data[0]["trans_world"][0], device=device).float()

# Split pose correctly for SMPLH
root_orient = wham_pose[:3].reshape(1, -1)     # [1, 3]
pose_body = wham_pose[3:66].reshape(1, -1)     # [1, 63]
hand_pose = torch.zeros(1, 90, device=device)  # [1, 90]

# Pass to BodyModel with padded betas
try:
    wham_body = bm(
        pose_body=pose_body,        
        root_orient=root_orient,    
        pose_hand=hand_pose,        
        trans=trans,                
        betas=wham_betas.view(1, -1)  # Now [1, 16]
    )
except RuntimeError as e:
    print("Error during BodyModel forward pass:", e)
    raise

# Create Trimesh object for WHAM mesh
wham_mesh = trimesh.Trimesh(
    vertices=wham_body.v.detach().cpu().numpy()[0],
    faces=bm.bm.faces_tensor.cpu().numpy()  # Changed from bm.f to bm.bm.faces_tensor
)

# Load and process HuMoR mesh
# Load HuMoR results
humor_res = np.load(HUMOR_OUT_FILE)

# Convert HuMoR parameters to tensors and pad betas to 16 dimensions
humor_betas = torch.tensor(humor_res["betas"][0], device=device).float()      # [10]
humor_betas = torch.cat([                                                     # Pad to [16]
    humor_betas,
    torch.zeros(6, device=device)
])
humor_trans = torch.tensor(humor_res["trans"][0], device=device).float()      # [3]
humor_pose = torch.tensor(humor_res["pose_body"][0], device=device).float()   # [63]
humor_root = torch.tensor(humor_res["root_orient"][0], device=device).float() # [3]

# Initialize zero hand pose
humor_hand_pose = torch.zeros(1, 90, device=device)  # [1, 90] for SMPLH hands

# Get SMPL output for HuMoR parameters with padded betas
humor_body = bm(
    pose_body=humor_pose.view(1, -1),     # [1, 63]
    root_orient=humor_root.view(1, -1),    # [1, 3] 
    pose_hand=humor_hand_pose,            # [1, 90]
    trans=humor_trans.view(1, -1),        # [1, 3]
    betas=humor_betas.view(1, -1)         # [1, 16] - now padded
)
# Create trimesh from HuMoR SMPL output
humor_mesh = trimesh.Trimesh(
    vertices=humor_body.v.detach().cpu().numpy()[0],
    faces=bm.bm.faces_tensor.cpu().numpy()
)

# Compare coordinate systems
print("WHAM Original:")
wham_orig = visualize_coordinate_system(wham_mesh, 'identity')

print("\nHuMoR Original:")
humor_orig = visualize_coordinate_system(humor_mesh, 'identity')

# Apply transformations
transforms = ['invert_z', 'invert_yz', 'swap_yz']
for transform in transforms:
    print(f"\nTrying {transform}:")
    visualize_coordinate_system(wham_mesh, transform)

In [4]:
def process_mesh_data(
    mesh_type: str,  # Just "wham" or "humor"
    input_file: str,
    bm: 'BodyModel', 
    device: torch.device
) -> trimesh.Trimesh:
    """Process mesh data for either WHAM or HuMoR outputs."""
    if mesh_type.lower() not in ["wham", "humor"]:
        raise ValueError("mesh_type must be either 'wham' or 'humor'")
        
    # Rest of function remains same but with string comparison
    if mesh_type.lower() == "wham":
        data = joblib.load(input_file)
        betas = torch.tensor(data[0]["betas"][0], device=device).float()
        pose = torch.tensor(data[0]["pose_world"][0], device=device).float()
        trans = torch.tensor(data[0]["trans_world"][0], device=device).float()
        beta_padding = 12  # Pad from 10 to 22
    elif mesh_type.lower() == "humor":
        data = np.load(input_file)
        betas = torch.tensor(data["betas"][0], device=device).float()
        pose = torch.tensor(data["pose_body"][0], device=device).float()
        trans = torch.tensor(data["trans"][0], device=device).float()
        root_orient = torch.tensor(data["root_orient"][0], device=device).float()
        beta_padding = 6  # Pad from 16 to 22
    else:
        raise ValueError("Unknown mesh type")

    # Common processing
    betas = torch.cat([betas, torch.zeros(beta_padding, device=device)])
    hand_pose = torch.zeros(1, 90, device=device)
    
    # Handle poses differently based on type
    if mesh_type == "wham":
        root_orient = pose[:3].reshape(1, -1)
        pose_body = pose[3:66].reshape(1, -1)
    else:
        root_orient = root_orient.view(1, -1)
        pose_body = pose.view(1, -1)
    
    # Get body output
    try:
        body = bm(
            pose_body=pose_body,
            root_orient=root_orient,
            pose_hand=hand_pose,
            trans=trans.view(1, -1),
            betas=betas.view(1, -1)
        )
    except RuntimeError as e:
        print(f"Error processing {mesh_type} mesh:", e)
        raise
        
    # Create and return mesh
    return trimesh.Trimesh(
        vertices=body.v.detach().cpu().numpy()[0],
        faces=bm.bm.faces_tensor.cpu().numpy()
    )


FOLDER_CHOSEN = "/home/NAS-mountpoint/kinect-omni-ego/2023-02-09/at-unis/lab/a04-2/capture0/"
HUMOR_MESH = os.path.join(FOLDER_CHOSEN, "out_capture0/results_out/final_results/stage3_results.npz")
BM_PATH = "/home/sid/Projects/humor/body_models/smplh/male/model.npz"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize BodyModel with 22 betas
bm = BodyModel(
    bm_path=BM_PATH,
    num_betas=22,  # Changed to match SMPLH shape space
    batch_size=1,
    model_type='smplh'
).to(device)

humor_mesh = process_mesh_data("humor", HUMOR_MESH, bm, device)

In [6]:
humor_mesh.show()

In [17]:
REPROJECTED_MESHES_FOLDER = "/home/NAS-mountpoint/kinect-omni-ego/2023-02-09/at-unis/lab/a04-2/capture0/out_capture0/reprojected/meshes/" # Reprojected meshes from HuMoR
reprojected_humor_meshes = os.listdir(REPROJECTED_MESHES_FOLDER)
first_mesh_file = reprojected_humor_meshes[2]
reprojected_mesh = trimesh.load_mesh(os.path.join(REPROJECTED_MESHES_FOLDER, first_mesh_file))
reprojected_mesh.show()